In [ ]:
import os
import glob
import json
import random
import subprocess
import shutil
from pprint import pprint
from ecg_datasets import ECGDataset
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import pandas as pd
import ast
import numpy as np
import wfdb
from sklearn.preprocessing import MultiLabelBinarizer
import torch
import hydra
from omegaconf import OmegaConf
from hpt.models.policy import Policy
import torch.nn as nn
import torch.optim as optim
import pickle


def print_ts(obj):
    print(type(obj))
    print(obj.shape)

In [ ]:
lead_names = ["I", "II", "III", "aVR", "aVL", "aVF", "V1", "V2", "V3", "V4", "V5", "V6"]
sampling_rate=100
start_time = 0
time = 10
start_length = int(start_time * sampling_rate)
sample_length = int(time * sampling_rate)
end_time = start_time + time
t = np.arange(start_time, end_time, 1 / sampling_rate)
data_path = '../../ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.3/'

In [ ]:
"""def draw_ecg(ecg, lead=1):
    print(type(ecg))
    print(ecg.shape)
    plt.plot(
        t,
        ecg[lead][start_length: start_length + sample_length],
        linewidth=2,
        color="k",
        alpha=1.0,
        label=lead_names[lead]
    )
    minimum = min(ecg[lead])
    maximum = max(ecg[lead])
    ylims_candidates = [-2.5, -2.0, -1.5, -1.0, -0.5, 0, 0.5, 1.0 , 1.5, 2.0, 2.5]

    ylims = (
        max([x for x in ylims_candidates if x <= minimum]),
        min([x for x in ylims_candidates if x >= maximum]),
    )
    plt.vlines(np.arange(start_time, end_time, 0.2), ylims[0], ylims[1], colors="r", alpha=1.0)
    plt.vlines(np.arange(start_time, end_time, 0.04), ylims[0], ylims[1], colors="r", alpha=0.3)
    plt.hlines(np.arange(ylims[0], ylims[1], 0.5), start_time, end_time, colors="r", alpha=1.0)
    plt.hlines(np.arange(ylims[0], ylims[1], 0.1), start_time, end_time, colors="r", alpha=0.3)

    plt.xticks(np.arange(start_time, end_time + 1, 1.0))
    plt.margins(0.0)
    plt.show()

def draw_ecgs(ecgs, lead=1):
    for i, ecg in enumerate(ecgs):
        plt.rcParams["figure.figsize"] = (25, 1.5 * len(ecgs))
        plt.subplot(len(ecgs), 1, i + 1)
        draw_ecg(ecg, lead)

def visualize_sample(sample_data, lead):
    ecgs = []
    for i, ecg_path in enumerate(sample_data["ecg_path"]):
        if not os.path.exists(ecg_path + ".dat"):
            download_ptbxl(sample_data["ecg_id"][i])

        ecg, _ = wfdb.rdsamp(ecg_path)
        ecgs.append(ecg.T)

    draw_ecgs(ecgs, lead)
    print(f"Sample ID: {sample_data['sample_id']}")
    print(f"Question: {sample_data['question']}")
    print(f"Answer: {sample_data['answer']}")

def get_ptbxl_data_path(ecg_id):
    return os.path.join(
        f"{int(ecg_id / 1000) * 1000 :05d}",
        f"{ecg_id:05d}_hr"
    )

def download_ptbxl(ecg_id):
    ptbxl_data_path = get_ptbxl_data_path(ecg_id)
    !wget -r -N -c np https://physionet.org/files/ptb-xl/1.0.3/records500/{ptbxl_data_path}.hea -P ptbxl &> /dev/null
    shutil.move(
        os.path.join("ptbxl", f"physionet.org/files/ptb-xl/1.0.3/records500/{get_ptbxl_data_path(ecg_id)}.hea"),
        os.path.join("ptbxl")
    )

    !wget -r -N -c np https://physionet.org/files/ptb-xl/1.0.3/records500/{ptbxl_data_path}.dat -P ptbxl &> /dev/null
    shutil.move(
        os.path.join("ptbxl", f"physionet.org/files/ptb-xl/1.0.3/records500/{get_ptbxl_data_path(ecg_id)}.dat"),
        os.path.join("ptbxl")
    )
    shutil.rmtree(os.path.join("ptbxl", "physionet.org"))"""

In [ ]:
"""def load_raw_data(df, sampling_rate, path):
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    data = np.array([signal for signal, meta in data])
    return data"""

In [ ]:
"""def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))"""

In [ ]:
#data_path = "/home/qfbqt/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.3/"

In [ ]:
"""# load and convert annotation data
Y = pd.read_csv(data_path+'ptbxl_database.csv', index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

# Load raw signal data
X = load_raw_data(Y, sampling_rate, data_path)

# Load scp_statements.csv for diagnostic aggregation
agg_df = pd.read_csv(data_path+'scp_statements.csv', index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]
Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)"""

In [ ]:
"""labels_array = Y['diagnostic_superclass'].values
#remove the entries without any superclass
mask = [len(labels) > 0 for labels in labels_array]
mask_array = np.array(mask)
ecgdata_filtered = X[mask_array]
labels_filtered = labels_array[mask_array]
# Create a LabelEncoder instance
label_encoder = MultiLabelBinarizer()
labels_filtered_encoded = label_encoder.fit_transform(labels_filtered)"""

In [ ]:
"""with open('ecgdata_filtered.pk', 'wb') as f:
    pickle.dump(ecgdata_filtered, f)

# Save labels_filtered_encoded to 'labels_filtered_encoded.pk'
with open('labels_filtered_encoded.pk', 'wb') as f:
    pickle.dump(labels_filtered_encoded, f)

print("Objects have been saved successfully.")"""

In [ ]:
ecgdata_filtered = None
labels_filtered_encoded = None

# Check and load ecgdata_filtered
if os.path.exists('ecgdata_filtered.pk'):
    with open('ecgdata_filtered.pk', 'rb') as f:
        ecgdata_filtered = pickle.load(f)
    print("Loaded ecgdata_filtered from file.")
else:
    print("File 'ecgdata_filtered.pk' does not exist.")

# Check and load labels_filtered_encoded
if os.path.exists('labels_filtered_encoded.pk'):
    with open('labels_filtered_encoded.pk', 'rb') as f:
        labels_filtered_encoded = pickle.load(f)
    print("Loaded labels_filtered_encoded from file.")
else:
    print("File 'labels_filtered_encoded.pk' does not exist.")

In [ ]:
ecg_dataset = ECGDataset(ecgdata_filtered, labels_filtered_encoded)
train_dataset, test_dataset, valid_dataset = torch.utils.data.random_split(ecg_dataset, [0.7, 0.2, 0.1])

In [ ]:
# Retrieve the normalizer to use for validation and test sets
normalizer = ecg_dataset.get_normalizer()


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False, num_workers=4)
"""
batch_size = 16  # Adjust as needed

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)"""

In [ ]:
cfg = OmegaConf.load('experiments/configs/config.yaml')
policy = Policy(embed_dim=128, num_blocks=4, num_heads=4, action_horizon=0)
# Assuming 'ecg_dataset' is your domain name
policy.init_domain_stem(domain_name='ecg_dataset', stem_spec=cfg.stem_ecg)
# Initialize domain head with the normalizer
policy.init_domain_head(
    domain_name='ecg_dataset',
    normalizer=normalizer,
    head_spec=cfg.head_ecg
)
policy.finalize_modules()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
policy.to(device)
pass

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(policy.parameters(), lr=cfg.optimizer.lr, weight_decay=cfg.optimizer.weight_decay)

In [ ]:
num_epochs = cfg.train.total_epochs

for epoch in range(num_epochs):
    policy.train()
    running_loss = 0.0
    for batch in train_loader:
        #inputs = batch['data'].to(device).float()  # Shape: (batch_size, sequence_length, input_dim)
        #labels = batch['label'].to(device).long()  # Shape: (batch_size,)
        
        #print_ts(inputs)
        #print_ts(labels)
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = policy.compute_loss(batch)  # Adjust based on your model's expected input
        
        # Compute loss
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    avg_loss = running_loss / len(train_loader)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}')
    
    # Optional: Validate the model
    # policy.eval()
    # with torch.no_grad():
    #     # Perform validation
